In [ ]:
:opt no-lint

# Improving and combining functions

## Outline

- Higher-order functions
	- `filter`
	- `any`
- Lambda functions
- Precedence and associativity
- Curried functions
    - Partial application
- Applying and composing functions 
	- The `$` operator
	- The `.` operator
- Point-free style

## Higher-order functions

Một **hàm có thứ tự ưu tiên cao hơn** **(higher-order function)** là một hàm mà dùng các hàm khác để nhận vào với vai trò các tham số hoặc kết quả trả về.

Vì chúng ta có thể đưa các hàm với vai trò là tham số đầu vào, trả chúng về như là kết quả, và gán chúng như các biến, chúng giống như bất kì gía trị nào khác. Cho nên chúng ta sẽ gọi những hàm đó là **first-class citizens**.

Chúng ta hãy bắt đầu với một ví dụ cơ bản sau. Hãy tưởng tượng bạn cáo một hàm mà bạn thường áp dụng hai lần (vì một vài nguyên do). Giống như sau:

In [ ]:
complexFunc1 :: Int -> Int
complexFunc1 x = x + 1

func1 :: Int -> Int
func1 x = complexFunc1 (complexFunc1 x)

complexFunc2 :: Int -> Int
complexFunc2 x = x + 2

func2 :: Int -> Int
func2 x = (complexFunc2 (complexFunc2 x)) + (complexFunc2 (complexFunc2 x))

: 

Đây là một ví dụ được làm phức tạp hóa, nhưng bạn có thể nhận thấy cách mà một ví dụ được thể hiện. Bạn luôn sử dụng `complexFunc1` và `complexFunc2` hai lần! Ngay khi mà chúng ta xem ví dụ này, chúng ta nhận ra rằng ta có thể thực hiện một cách tốt hơn. Sẽ ra sao nếu như ta tạo một hàm mà nhận vào hai tham số - một hàm và một giá trị - và thực thi hàm đó hai lần với giá trị đã nhận vào!

Ta có thể thực hiện điều đó như sau:

In [ ]:
applyTwice :: (a -> a) -> a -> a
applyTwice f x = f (f x)

Ở đây, chữ kí cho kiểu dữ liệu sẽ khác so với ví dụ trước đó. Phần `(a -> a)` cho thấy tham số đầu tiên là một hàm nhận vào giá trị với kiểu `a` và trả về giá trị cùng kiểu dữ liệu. Tham số thứ hai chỉ là một giá trị với kiểu `a`, và cả hai hàm `applyTwice` đều trả về một giá trị kiểu `a`.

Ta có thể sử dụng hàm `applyTwice` để đơn giản hóa code trước đó như sau:

In [ ]:
func1' :: Int -> Int
func1' x = applyTwice complexFunc1 x

func2' :: Int -> Int
func2' x = (applyTwice complexFunc2 x) + (applyTwice complexFunc2 x)

Trên đây là ví dụ đơn giản, nhưng **các hàm với thứ tự ưu tiên cao hơn** là một tính năng đầy mạnh mẽ. Vì thế chúng xuất hiện ở khắp nơi. Thực tế, bạn có thể tạo ra **Domain Specific Language** của riêng bạn sử dụng **higher-order functions**! Nhưng hãy đi lần lượt từng bước. Giờ hãy bắt đầu bằng cách sử dụng 2 **higher-order functions** tích hợp trong Haskell.

### Hàm `filter`

Hãy bắt đầu với hàm `filter`:

In [ ]:
:t filter 

Hàm này nhận vào một hàm thuộc tính (một hàm trả về một số kiểu boolean) `a -> Bool` và một mảng các phần tử kiểu `a` và lọc những phần tử của mảng thỏa hàm kiểm tra thuộc tính.

Ví dụ, nếu ta muốn lọc chỉ những số chẵn trong mảng các phần tử từ 1 đến 20, chúng ta có thể code một vài thứ như sau:

In [ ]:
filter even [1..20]

Hoặc là, để dùng các điều kiện khác, ta có thể lọc từ một mảng trái cây chỉ những trái mà bao gồm kí tự `'a'`:

In [ ]:
fruitWithA = filter tempFunct ["Apple", "Banana", "Pear", "Grape", "Wood"]
                where tempFunct x = 'a' `elem` x
fruitWithA

Như bạn có thể thấy, bạn cũng có thể định nghĩa trong một mệnh đề `where` để bỏ vào hàm kiểm tra trong một hàm `filter` function.

### Hàm `any`

Ta có hàm `any`:
- Hàm này nhận vào một hàm kiểm tra và một mảng các phần tử. Nhưng hàm này kiểm tra nếu tồn tại bất kì phần tử nào trong mảng thỏa hàm kiểm tra.
- Ví dụ, ở đây ta kiểm tra xem có bất kì phần tử nào trong mảng lớn hơn 4. Nếu có bất kì phần tử nào thỏa thì trả về `True`, ngược lại trả về `False`:

In [ ]:
biggerThan4 x = x > 4

any biggerThan4 [1,2,3,4] 

Một ví dụ thực tế về việc sử dụng hàm `any` là khi ta kiểm tra xem còn xe nào ở trên trang web bán xe:

In [ ]:
cars = [("Toyota",0), ("Nissan",3), ("Ford",1)]

biggerThan0 (_,x) = x > 0

any biggerThan0 cars

## Hàm `Lambda`

Hàm `lambda` được bắt nguồn từ phép tính trong toán học gọi là `phép tính lambda`. Bản thân nó là một chủ đề hấp dẫn và là một công cụ vô cùng mạnh mẽ trong **Calculus**. Tuy nhiên, tại đây, ta chỉ nhìn nhận nó với góc độ của việc lập trình.

Hàm `lambda` còn được gọi là hàm ẩn danh hay hàm mà được định nghĩa mà không hề có tên.

Ví dụ, hàm `f(x, y) = x * y` trông như sau trong Haskell:

```haskell
\x y -> x * y
```

Một hàm `lambda` bao gồm 4 thành phần sau: 
- Bắt đầu với một dấu gạch chéo ngược `\` để đánh dấu đây là hàm `lambda`.
- Tên các tham số (trong trường hợp này là x và y) mà hàm nhận làm đầu vào.
- Mũi tên (`->`) có tác dụng **tách** phần đầu vào và phần thân hàm.
- Và mọi thứ sau dấu mũi tên sẽ là **phần thân** của hàm.

## Precedence and associativity (Tính ưu tiên và Tính kết hợp)

### Precedence (Tính ưu tiên)

**Tính ưu tiên** biểu hiện độ ưu tiên của một toán tử (được đánh dấu bởi các con số từ 0 đến 9). Nếu chúng ta sử dụng hai toán tử với độ ưu tiên khác nhau, toán tử có độ ưu tiên cao hơn sẽ được áp dụng trước. Nghĩa là toán thử có độ ưu tiên cao hơn thì ràng buộc chặt chẽ hơn!

Ta có thể lấy độ ưu tiên của toán tử bằng hàm lấy thông tin `:i`.

In [ ]:
:i (+)  -- infixl 6 +
:i (*)  -- infixl 7 *

1 + 2 * 3  -- Same as 1 + (2 * 3)

<div class="alert alert-block alert-info">
    <code>infixl 6 +</code> và <code>infixl 7 *</code> được gọi là <b>fixity declarations</b>.
</div>

Thế nhưng sẽ ra sao nếu 2 toán tử có cùng độ ưu tiên ? Lúc này **tính kết hợp** sẽ được xét đến.

### Associativity (Tính kết hợp)

Khi chúng ta dùng lệnh `:i` bên trên, ta nhạn được từ khóa `infixl`. Đây là tính kết hợp của toán tử.

Khi hai toán tử có cùng độ ưu tiên thì tính kết hợp sẽ chỉ ra cho ta biết hướng nào (`infixl` tức bên trái hay `infixr` tức bên phải) sẽ được tính toán trước.

Ví dụ:
- Toán tử (+) và (*) thì có tính kết hợp bên trái, tức là sẽ được tính toán phía bên trái trước.
- Toán tử (:) thì có tính kết hợp là bên phải, nên sẽ được tính từ phải trước.
- Toán tử (==) thì không có tính kệp hợp `infix`, tức nghĩa là, nếu bạn dùng nhiều hơn 1 lần toán tử này thì phải dùng các dấu ngoặc đơn để chỉ rõ thứ tự thực thi.

In [ ]:
1 + 2 + 3 + 4  -- infixl: Same as ((1 + 2) + 3) + 4

1 : 2 : 3 : [] -- infixr: Same as 1 : (2 : (3 : []))

True == (False == False) -- infix: If you remove parenthesis, you'll get an error.

Và, đương nhiên, ta có thể thay đổi thứ tự thực thi bằng cách sử dụng các cặp ngoặc đơn:

In [ ]:
:i (**) -- infixr 8 **

2**3**4  -- infixr: Same as 2 ** (3 ** 4)
(2**3)**4

Cuối cùng, ta hoàn toàn có thể tự định nghĩa độ phức tạp và tính kết hợp của toán từ mà ta tự định nghĩa. Như sau:

In [ ]:
x +++ y = x + y -- Creating +++ operator
infixl 7 +++    -- Setting fixity of operator

1 +++ 2 * 3  -- 9

<div class="alert alert-block alert-info">
<b>Important note:</b> 
   <ul>
       <li>Các toán tử mà không có định nghĩa về fixity thì mặc định là <code>infixl 9</code></li>
       <li>Việc thực thi gọi hàm (toán tử `whitespace`) luôn có độ ưu tiên cao nhất (hãy hình dung độ ưu tiên là 10).</li>
   </ul>
</div>